In [16]:
import enum

import isort
import sys
import platform
import importlib
from types import ModuleType
import inspect

In [17]:
def get_stdlib_packages():
    if sys.version_info.minor == 10:
        module_names = sys.stdlib_module_names
    else:
        module_names = isort.stdlibs.py38.stdlib

    external_packages = list()
    for name in module_names:
        if name[0] == '_' or name == 'this' or name == 'antigravity':
            continue
        external_packages.append(name)
    return external_packages

def get_real_packages(package_names):
    real_modules = list()
    not_importable_modules = list()
    for name in package_names:
        try:
            importlib.import_module(name)
            real_modules.append(name)
        except:
            not_importable_modules.append(name)
    return real_modules, not_importable_modules


def get_real():
    external_packages = get_stdlib_packages()
    return get_real_packages(external_packages)

In [18]:
def module_dependency(module_names, name):
    if name not in module_names:
        raise Exception(f'{name} is not importable module')
    dp_names = list()

    # try:
    #     importlib.import_module(name)
    # except:
    #     print(f'err name: {name}')
    for key, val in vars(sys.modules[name]).items():
        if isinstance(val, ModuleType):
            md_name = val.__name__

            try:
                index = md_name.index(".")
                md_name = md_name[0:index]
            except:
                pass

            dp_names.append(md_name)
            # print(f'key: {key}, type: {type(val)}, val: {md_name}, module: {md_name}')

    return dp_names

In [19]:
real_modules, _ = get_real()
print(real_modules)

['subprocess', 'sqlite3', 'smtplib', 'xdrlib', 'code', 'copyreg', 'wave', 'ast', 'calendar', 'nntplib', 'uuid', 'mimetypes', 'string', 'abc', 'pathlib', 'zipimport', 'cgitb', 'shutil', 'cmath', 'getpass', 'grp', 'shelve', 'platform', 'http', 'venv', 'formatter', 'random', 'cProfile', 'sched', 'pydoc', 'builtins', 'py_compile', 'fnmatch', 'hashlib', 'bisect', 'colorsys', 'imaplib', 'numbers', 'urllib', 'webbrowser', 'token', 'typing', 'array', 'fcntl', 'sys', 'io', 'stat', 'hmac', 'xmlrpc', 'xml', 'pickle', 'datetime', 'distutils', 'bdb', 'posix', 'posixpath', 'dbm', 'cmd', 'os', 'smtpd', 'mailcap', 'profile', 'reprlib', 'asyncore', 'base64', 'types', 'ipaddress', 'email', 'trace', 'collections', 'site', 'tty', 'importlib', 'concurrent', 'linecache', 'pyclbr', 'zlib', 'resource', 'compileall', 'contextvars', 'logging', 'mmap', 'pkgutil', 'socketserver', 'aifc', 'codeop', 'chunk', 'wsgiref', 'runpy', 'pty', 'faulthandler', 'ctypes', 'heapq', 'dummy_threading', 'imghdr', 'imp', 'inspect',

In [20]:
import pathlib

dp_names = module_dependency(real_modules, 'pathlib')
print(dp_names)

['fnmatch', 'functools', 'io', 'ntpath', 'os', 'posixpath', 're', 'sys']


In [21]:
# import sys
# sys.setrecursionlimit(50000)
#
# def dependency_graph(modules, ad_mtrx, original_modules, count):
#     count = count + 1
#     if count % 10 == 0:
#         print(f'count: {count}')
#     for md in modules:
#         try:
#             dp_names = module_dependency(original_modules, md)
#         except:
#             print(f'{md} is not in stdlib')
#             continue
#
#         for name in dp_names:
#             try:
#                 i = original_modules.index(md)
#                 j = original_modules.index(name)
#                 ad_mtrx[i][j] = 1
#             except:
#                 print(f'md: {md}, name: {name} is not in stdlib')
#                 continue
#
#         dependency_graph(dp_names, ad_mtrx, original_modules, count)
#
# n = len(real_modules)
# ad_mtrx = [[0 for x in range(n)] for y in range(n)]
# dependency_graph(real_modules, ad_mtrx, real_modules, 0)
# print(ad_mtrx)

In [22]:
dp = module_dependency(real_modules, 'os')
print(dp)

['abc', 'sys', 'stat', 'posixpath']


In [23]:
def module_dependency_map(modules):
    md_map = dict()
    for md in modules:
        md_map[md] = module_dependency(modules, md)
    return md_map

md_map = module_dependency_map(real_modules)
print(md_map)

{'subprocess': ['builtins', 'errno', 'io', 'os', 'time', 'signal', 'sys', 'threading', 'warnings', 'contextlib', '_posixsubprocess', 'select', 'selectors'], 'sqlite3': ['sqlite3', 'datetime', 'time', 'collections'], 'smtplib': ['socket', 'io', 're', 'email', 'base64', 'hmac', 'copy', 'datetime', 'sys', 'ssl'], 'xdrlib': ['struct'], 'code': ['sys', 'traceback'], 'copyreg': [], 'wave': ['builtins', 'audioop', 'struct', 'sys', 'warnings'], 'ast': [], 'calendar': ['sys', 'datetime', 'locale'], 'nntplib': ['re', 'socket', 'collections', 'datetime', 'warnings', 'sys', 'ssl'], 'uuid': ['os', 'sys', 'platform', '_uuid'], 'mimetypes': ['os', 'sys', 'posixpath', 'urllib'], 'string': ['_string', 're'], 'abc': [], 'pathlib': ['fnmatch', 'functools', 'io', 'ntpath', 'os', 'posixpath', 're', 'sys'], 'zipimport': ['importlib', 'importlib', '_imp', 'io', 'marshal', 'sys', 'time'], 'cgitb': ['inspect', 'keyword', 'linecache', 'os', 'pydoc', 'sys', 'tempfile', 'time', 'tokenize', 'traceback'], 'shutil':

In [29]:
def build_dependency_graph(md_map):
    index_list = list()

    for k, v in md_map.items():
        index_list.append(k)

    n = len(index_list)
    ad_mtrx = [[0 for x in range(n)] for y in range(n)]

    for md in index_list:
        for name in md_map[md]:
            try:
                i = index_list.index(md)
                j = index_list.index(name)
                ad_mtrx[i][j] = 1
            except:
                continue

    return index_list, ad_mtrx


index_list, ad_mtrx = build_dependency_graph(md_map)
# print(f'index list: {index_list}')
# print(f'ad_mtrx: {ad_mtrx}')

i = index_list.index('locale')
print(f'dp of locale ---------')
dp_list = list()
for j in range(len(index_list)):
    if ad_mtrx[i][j] == 1:
        dp_list.append(index_list[j])
print(dp_list)

dp of locale ---------
['sys', 'collections', 'functools', 're', 'encodings']


In [30]:
dp = module_dependency(real_modules, 'locale')
print(dp)

['sys', 'encodings', 're', 'collections', 'functools']
